In [30]:
import requests
import pandas as pd
from time import sleep
class CkanConsumer:
    def __init__(self, main_url:str, secure:bool = True):
        if secure:
            self.url = f'https://{main_url}/api/action/datastore_search'
        else:
            self.url = f'http://{main_url}/api/action/datastore_search'

    def request(self, resource_id) -> pd.DataFrame:
        params = {
            'limit': 1000,
            'offset': 0,
            'resource_id': resource_id
        }
        data = []

        print(f'[INFO] - Getting data from {self.url} on resource {resource_id} at offset {len(data)}')
        response = requests.get(self.url, params=params)
        total = response.json()['result']['total']
        data.extend(response.json()['result']['records'])

        while len(data) < total:
            params['offset'] = len(data)
            print(f'[INFO] - Getting data from {self.url} on resource {resource_id} at offset {len(data)}')
            response = requests.get(self.url, params=params)
            while response.status_code != 200:
                print(f'[ERROR] - Response code {response.status_code} from {self.url} on resource {resource_id} at offset {len(data)}')
                print(f'[INFO] - Waiting 5 seconds before trying again')
                sleep(5)
                response = requests.get(self.url, params=params)
            data.extend(response.json()['result']['records'])
        
        print(f'[INFO] - Total of {len(data)} records retrieved from {self.url} on resource {resource_id}')
        #Reassembling the dataframe
        df = pd.DataFrame(data)
        return df

In [32]:
# Use ckan_consumer.request(resource_id) to get a dataframe from a resource_id
ufrn_consumer = CkanConsumer('dados.ufrn.br')
cursos_ufrn = ufrn_consumer.request('a10bc434-9a2d-491a-ae8c-41cf643c35bc')
docentes_ufrn = ufrn_consumer.request('6a8e5461-e748-45c6-aac6-432188d88dde')
discentes_ufrn = ufrn_consumer.request('14afbb6c-395e-411c-b24d-0e494cb95866')

[INFO] - Getting data from https://dados.ufrn.br/api/action/datastore_search on resource a10bc434-9a2d-491a-ae8c-41cf643c35bc at offset 0
[INFO] - Total of 120 records retrieved from https://dados.ufrn.br/api/action/datastore_search on resource a10bc434-9a2d-491a-ae8c-41cf643c35bc
[INFO] - Getting data from https://dados.ufrn.br/api/action/datastore_search on resource 6a8e5461-e748-45c6-aac6-432188d88dde at offset 0
[INFO] - Getting data from https://dados.ufrn.br/api/action/datastore_search on resource 6a8e5461-e748-45c6-aac6-432188d88dde at offset 1000
[INFO] - Getting data from https://dados.ufrn.br/api/action/datastore_search on resource 6a8e5461-e748-45c6-aac6-432188d88dde at offset 2000
[INFO] - Total of 2772 records retrieved from https://dados.ufrn.br/api/action/datastore_search on resource 6a8e5461-e748-45c6-aac6-432188d88dde
[INFO] - Getting data from https://dados.ufrn.br/api/action/datastore_search on resource 14afbb6c-395e-411c-b24d-0e494cb95866 at offset 0
[INFO] - Gettin

In [50]:
docentes_ufrn.head()

,categoria,vinculo,nome,siape,admissao,formacao,sexo,id_unidade_lotacao,lotacao,tipo_jornada_trabalho,_id,classe_funcional,ano_ingresso,periodo_ingresso
0,PROFESSOR DE ENSINO BASICO TECNICO E TECNOLOGICO,Ativo Permanente,ADELE GUIMARAES UBARANA SANTOS,1543339,2006-07-24,MESTRADO,F,1452,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA,Dedicação exclusiva,1,DIV ...,2006,2
1,PROFESSOR DE ENSINO BASICO TECNICO E TECNOLOGICO,Ativo Permanente,AFRANIO CESAR DE ARAUJO,1554468,2008-09-12,DOUTORADO,M,351,ESCOLA AGRÍCOLA DE JUNDIAÍ,Dedicação exclusiva,2,DV ...,2008,2
2,PROFESSOR DE ENSINO BASICO TECNICO E TECNOLOGICO,Ativo Permanente,AIRTON FERNANDES GUIMARAES FILHO,1177821,1998-04-28,MESTRADO,M,284,ESCOLA DE MÚSICA,Dedicação exclusiva,3,DIV ...,1998,2
3,PROFESSOR DE ENSINO BASICO TECNICO E TECNOLOGICO,Ativo Permanente,ALDAIR RODRIGUES DA SILVA,2360824,2017-01-25,MESTRADO,M,351,ESCOLA AGRÍCOLA DE JUNDIAÍ,Dedicação exclusiva,4,DIII ...,2017,1
4,PROFESSOR DE ENSINO BASICO TECNICO E TECNOLOGICO,Ativo Permanente,ALESSANDRA MENDES PACHECO,2364334,2009-10-13,DOUTORADO,F,351,ESCOLA AGRÍCOLA DE JUNDIAÍ,Dedicação exclusiva,5,DIV ...,2009,2


In [49]:
def trunc_date(df: pd.DataFrame, coluna_datetime) -> pd.DataFrame:
    docentes_ufrn[coluna_datetime] = pd.to_datetime(docentes_ufrn[coluna_datetime], format='%Y.%m')
    docentes_ufrn['ano_ingresso'] = docentes_ufrn[coluna_datetime].dt.year
    docentes_ufrn['periodo_ingresso'] = docentes_ufrn[coluna_datetime].dt.month
    docentes_ufrn['periodo_ingresso'] = docentes_ufrn['periodo_ingresso'].apply(lambda x: 1 if x == 1 else 2)

In [34]:
from hashlib import md5
treat = discentes_ufrn.apply(lambda df: pd.Series({
    'nome': str(df['nome_discente']).upper(),
    'id': md5((str(df['matricula'])+'ufrn'+'14afbb6c-395e-411c-b24d-0e494cb95866').encode()).hexdigest(),
    'codigo': str(df['id_curso']),
    'sexo': df['sexo'],
    'data_ingresso': f"{str(df['ano_ingresso'])}.{str(df['periodo_ingresso'])}",
    'curso': df['nome_curso']
}), axis=1)


In [38]:
#do the above cell like a function
def mapper_cursos(
        dataframe: pd.DataFrame,
        nome_curso: str,
        id_curso: str,
        instituto: str,
        resource_id: str
    ) -> pd.DataFrame:
    result = dataframe.apply(lambda df: pd.Series({
        'nome': str(df[nome_curso]).upper(),
        'id': md5((f"{df[nome_curso]}{instituto}{resource_id}").encode()).hexdigest(),
        'codigo': str(df[id_curso]),
        'instituicao': instituto
    }), axis = 1)
    return result

df = mapper_discentes(discentes_ufrn, 'nome_discente', 'matricula', 'sexo', 'ano_ingresso', 'periodo_ingresso', 'nome_curso', 'ufrn', '14afbb6c-395e-411c-b24d-0e494cb95866')

In [39]:
df.head()

,nome,id,matricula,sexo,data_ingresso,curso
0,AARON VINICIUS MAIA SOBRINHO,6f91a90b48840e78a5b05a5bdcc39283,20220017174,M,2022.1,ENGENHARIA DE PETRÓLEO
1,ABÃ BARBOSA DA SILVA,067ebfd1e4d331f9e707e343462954df,20220057240,M,2022.2,ENGENHARIA MECÂNICA
2,ABDA BEATRIZ DE ARAUJO PINHEIRO,51bdd617924f6b3c8be29838786fba55,20220046390,F,2022.1,BACHARELADO INTERDISCIPLINAR EM HUMANIDADES
3,ABDALLAH SAID SALEH YUSUF,1461a3cb16dd3a16ab9d378f921cc8fa,20220077682,M,2022.2,
4,ABEL BARBOSA DE SOUZA NETO,c02153fcda7e2cfa08087d1600e66c5a,20223009354,M,2022.1,CURSO TÉCNICO DA METRÓPOLE DIGITAL
